<a href="https://colab.research.google.com/github/PrabhashDiss/Gemma/blob/main/Merge_Large_Language_Models_with_mergekit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.aiplanet.com/create-your-own-mixture-of-experts-model-with-mergekit-and-runpod-8b3e91fb027a

In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .


Cloning into 'mergekit'...
remote: Enumerating objects: 1271, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 1271 (delta 568), reused 475 (delta 435), pack-reused 572
Receiving objects: 100% (1271/1271), 356.10 KiB | 4.68 MiB/s, done.
Resolving deltas: 100% (859/859), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 12.5 MB/s eta 0:00:00
  Building editable for mergekit (pyproject.toml) ... done


In [2]:
import yaml

MODEL_NAME = "Moe-4x7b-math-reason-code"
yaml_config = """
base_model: mistralai/Mistral-7B-Instruct-v0.2
dtype: float16
gate_mode: cheap_embed
experts:
  - source_model: HuggingFaceH4/zephyr-7b-beta
    positive_prompts: ["You are an helpful general-pupose assistant."]
  - source_model: mistralai/Mistral-7B-Instruct-v0.2
    positive_prompts: ["You are helpful assistant."]
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [3]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [4]:
import wandb
wandb.init()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
# Merge models
!mergekit-moe config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle --trust-remote-code


Streaming output truncated to the last 5000 lines.
model-00005-of-00008.safetensors:  30% 598M/1.98G [00:17<00:45, 30.6MB/s]







model-00008-of-00008.safetensors:  67% 545M/816M [00:17<00:07, 34.7MB/s]

model-00006-of-00008.safetensors:  31% 598M/1.95G [00:18<00:35, 38.2MB/s]






model-00001-of-00008.safetensors:  29% 556M/1.89G [00:17<00:37, 35.1MB/s]




model-00002-of-00008.safetensors:  30% 587M/1.95G [00:18<00:40, 33.5MB/s]








model-00004-of-00008.safetensors:  30% 577M/1.95G [00:17<00:35, 38.6MB/s]


model-00007-of-00008.safetensors:  31% 608M/1.98G [00:18<00:41, 33.3MB/s]





model-00003-of-00008.safetensors:  30% 587M/1.98G [00:18<00:40, 34.5MB/s]

model-00006-of-00008.safetensors:  31% 608M/1.95G [00:18<00:34, 39.0MB/s]



model-00005-of-00008.safetensors:  31% 608M/1.98G [00:18<00:43, 31.7MB/s]







model-00008-of-00008.safetensors:  68% 556M/816M [00:18<00:08, 32.2MB/s]






model-00001-of-00008.safetensors:  30% 566M/1.89G [00:18<00:38, 34.7MB/s]








model

In [6]:
# Stop tracking
wandb.finish()


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
            "merge/", device_map='cpu', torch_dtype=torch.float16
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [12]:
from huggingface_hub import notebook_login

notebook_login()


In [29]:
model.push_to_hub("Prabhash/Moe-4x7b-math-reason-code")
tokenizer.push_to_hub("Prabhash/Moe-4x7b-math-reason-code")


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })